T.U.I.A 2024 - Aprendizaje Automático II 

TP 1 - Redes Densas y Convolucionales

Ponce, Daniel - Yañez, Mirian

Problema 1

Instalamos las dependencias

In [1]:
!pip install gdown
!pip install pandas
!pip install tensorflow
!pip install scikit-learn
!pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


Importamos las librerías

In [ ]:
import gdown
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import zipfile
import matplotlib.pyplot as plt

In [3]:
class RegressionModel:
    def __init__(self):
        self.dataset = None
        self.model = None
    
    def download_dataset(self, file_id, output_name="dataset.zip"):
        """Descarga el dataset desde Google Drive."""
        url = f"https://drive.google.com/uc?id={file_id}"
        gdown.download(url, output_name, quiet=False)
        print(f"Descargado: {output_name}")

    def extract_zip(self, zip_path="dataset.zip", extract_to="."):
        """Extrae el contenido del archivo ZIP."""
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Contenido extraído en: {extract_to}")

    def load_dataset(self, path="student_performance.csv"):
        """Carga el dataset en un DataFrame."""
        self.dataset = pd.read_csv(path, encoding='utf-8')
        print(self.dataset.head())
        return self.dataset

    def preprocess_data(self):
        """Preprocesa los datos para el entrenamiento."""
        data = self.dataset.copy()

        # Codificación de actividades extracurriculares (Sí/No)
        label_encoder = LabelEncoder()
        data['Extracurricular Activities'] = label_encoder.fit_transform(
            data['Extracurricular Activities']
        )

        # Separación de características y variable objetivo
        X = data.drop('Performance Index', axis=1)
        y = data['Performance Index']

        # Escalado de las características
        scaler = StandardScaler()
        X = scaler.fit_transform(X)

        return train_test_split(X, y, test_size=0.2, random_state=42)

    def build_model(self, input_dim):
        """Construye el modelo usando Input() en lugar de input_shape."""
        self.model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(input_dim,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(1)  # Capa de salida
        ])
        self.model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    def train_model(self, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
        """Entrena el modelo y devuelve el historial."""
        history = self.model.fit(
            X_train, y_train, validation_data=(X_val, y_val),
            epochs=epochs, batch_size=batch_size, verbose=1
        )
        self.plot_training_history(history)
        return history

    def plot_training_history(self, history):
        """Grafica las métricas de entrenamiento y validación."""
        plt.figure(figsize=(12, 5))

        # Pérdida (Loss)
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Training vs Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        # MAE (Error Absoluto Medio)
        plt.subplot(1, 2, 2)
        plt.plot(history.history['mae'], label='Training MAE')
        plt.plot(history.history['val_mae'], label='Validation MAE')
        plt.title('Training vs Validation MAE')
        plt.xlabel('Epoch')
        plt.ylabel('MAE')
        plt.legend()

        plt.show()

    def evaluate_model(self, X_train, y_train, X_test, y_test):
        """Evalúa el modelo en entrenamiento y test."""
        # Evaluación en entrenamiento
        y_train_pred = self.model.predict(X_train).flatten()
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_mae = mean_absolute_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        print(f"Entrenamiento - MSE: {train_mse}, MAE: {train_mae}, R²: {train_r2}")

        # Evaluación en test
        y_test_pred = self.model.predict(X_test).flatten()
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        print(f"Test - MSE: {test_mse}, MAE: {test_mae}, R²: {test_r2}")


In [4]:
file_id = "1mfpXVLqDJah-sO0CF29LjKUz5NtKjZqc"
model = RegressionModel()
model.download_dataset(file_id)
model.extract_zip()

Downloading...
From: https://drive.google.com/uc?id=1mfpXVLqDJah-sO0CF29LjKUz5NtKjZqc
To: /Users/miri/AAII-2/dataset.zip
100%|██████████| 49.7k/49.7k [00:00<00:00, 650kB/s]

Descargado: dataset.zip
Contenido extraído en: .


In [5]:
data


,Hours Studied,Previous Scores,Extracurricular Activities,Sleep Hours,Sample Question Papers Practiced,Performance Index
0,7,99,Yes,9,1,91.0
1,4,82,No,4,2,65.0
2,8,51,Yes,7,2,45.0
3,5,52,Yes,5,2,36.0
4,7,75,No,8,5,66.0
...,...,...,...,...,...,...
9995,1,49,Yes,4,2,23.0
9996,7,64,Yes,8,5,58.0
9997,6,83,Yes,8,5,74.0
9998,9,97,Yes,7,0,95.0


In [10]:
data.describe()

,Hours Studied,Previous Scores,Sleep Hours,Sample Question Papers Practiced,Performance Index
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,4.992900,69.445700,6.530600,4.583300,55.224800
std,2.589309,17.343152,1.695863,2.867348,19.212558
min,1.000000,40.000000,4.000000,0.000000,10.000000
25%,3.000000,54.000000,5.000000,2.000000,40.000000
50%,5.000000,69.000000,7.000000,5.000000,55.000000
75%,7.000000,85.000000,8.000000,7.000000,71.000000
max,9.000000,99.000000,9.000000,9.000000,100.000000


In [11]:
data.isnull().sum()

Hours Studied                       0
Previous Scores                     0
Extracurricular Activities          0
Sleep Hours                         0
Sample Question Papers Practiced    0
Performance Index                   0
dtype: int64

In [ ]:
data = model.load_dataset("student_performance.csv")
X_train, X_test, y_train, y_test = model.preprocess_data()
model.build_model(input_dim=X_train.shape[1])
history = model.train_model(X_train, y_train, X_test, y_test)
model.evaluate_model(X_train, y_train, X_test, y_test)